# LightGBM Modeling & Hyperparameter Tuning

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
import mlflow, mlflow.lightgbm
from sklearn.metrics import mean_absolute_error

In [3]:
# Load preprocessed data
train_df = pd.read_parquet('../data/processed/transformed_train.parquet')
valid_df = pd.read_parquet('../data/processed/transformed_valid.parquet')

In [4]:
# Define target and features
target = 'log_loss'
X_train, Y_train = train_df.drop(columns=['loss', target]), train_df[target]
X_valid, Y_valid = valid_df.drop(columns=['loss', target]), valid_df[target]

print('Train shape : ', X_train.shape)
print('Valid shape : ', X_valid.shape)

Train shape :  (150655, 5548)
Valid shape :  (18831, 5548)


In [ ]:
# Prepare lightgbm dataset
#train_lgb = lgb.Dataset(X_train, label=Y_train)
#valid_lgb = lgb.Dataset(X_valid, label=Y_valid)

In [5]:
# Define Optuna task (with MLflow)

def objective(trial):
    
    params = {
        'objective': 'regression',
        'metric': 'mae',

        'num_leaves': trial.suggest_int('num_leaves', 64, 512, log=True),
        'max_depth': trial.suggest_int('max_depth', 6, 14),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 200, 3000, log=True),

        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),

        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),

        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 50.0),

        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0,  5.0),

        'max_bin': trial.suggest_categorical('max_bin', [127, 255]),

        'boosting_type': 'gbdt',
        'verbosity': -1,
        'seed': 42,
    }

    train_lgb = lgb.Dataset(X_train, label=Y_train, params=params)
    valid_lgb = lgb.Dataset(X_valid, label=Y_valid, params=params, reference=train_lgb)

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)

        model = lgb.train(
            params,
            train_lgb,
            valid_sets = [valid_lgb],
            num_boost_round=10000,
            callbacks = [
                lgb.early_stopping(stopping_rounds=200),
                lgb.log_evaluation(period=100),
            ],
        )

        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
        mae = mean_absolute_error(np.exp(Y_valid), np.exp(y_pred))  # Calculate MAE based on true loss
        mlflow.log_params(params)
        mlflow.log_metric('mae', mae)
        mlflow.log_metric('best_iteration', model.best_iteration)

    return mae           

In [6]:
# Run Optuna study

# Use mlruns folder for MLflow results
mlflow.set_tracking_uri("../mlruns")

# Set and start experiment
mlflow.set_experiment('lgbm_optuna_allstate')

study = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.TPESampler(seed=42),
)

#study.optimize(objective, timeout=3*60*60)  # Run for 3 hours
study.optimize(objective, n_trials=20)

print('Best MAE: ', study.best_value)
print('Best params: ', study.best_params)

/Users/williamchen/projects/kaggle/Allstate_Claims_Severity/env/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/16 20:52:12 INFO mlflow.tracking.fluent: Experiment with name 'lgbm_optuna_allstate' does not exist. Creating a new experiment.
[I 2025-12-16 20:52:12,812] A new study created in memory with name: no-name-0e5b2434-416c-4c68-96f4-0595f99e933c


Training until validation scores don't improve for 200 rounds
[100]	valid_0's l1: 0.47046
[200]	valid_0's l1: 0.441336
[300]	valid_0's l1: 0.433231
[400]	valid_0's l1: 0.429583
[500]	valid_0's l1: 0.427568
[600]	valid_0's l1: 0.426355
[700]	valid_0's l1: 0.425624
[800]	valid_0's l1: 0.425188
[900]	valid_0's l1: 0.424698
[1000]	valid_0's l1: 0.424349
[1100]	valid_0's l1: 0.424159
[1200]	valid_0's l1: 0.423939
[1300]	valid_0's l1: 0.423804
[1400]	valid_0's l1: 0.423721
[1500]	valid_0's l1: 0.423674
[1600]	valid_0's l1: 0.423556
[1700]	valid_0's l1: 0.423512
[1800]	valid_0's l1: 0.423343
[1900]	valid_0's l1: 0.423348
[2000]	valid_0's l1: 0.423301
[2100]	valid_0's l1: 0.423247
[2200]	valid_0's l1: 0.42325
[2300]	valid_0's l1: 0.423199
[2400]	valid_0's l1: 0.423221
Early stopping, best iteration is:
[2279]	valid_0's l1: 0.423174


[I 2025-12-16 20:57:43,779] Trial 0 finished with value: 1162.2211620581852 and parameters: {'num_leaves': 139, 'max_depth': 14, 'min_data_in_leaf': 1451, 'learning_rate': 0.019843966652221864, 'feature_fraction': 0.5624074561769746, 'feature_fraction_bynode': 0.5779972601681014, 'bagging_fraction': 0.6174250836504598, 'bagging_freq': 9, 'lambda_l1': 6.011150117432088, 'lambda_l2': 35.40362888980228, 'min_gain_to_split': 0.10292247147901223, 'max_bin': 127}. Best is trial 0 with value: 1162.2211620581852.


Training until validation scores don't improve for 200 rounds
[100]	valid_0's l1: 0.51048
[200]	valid_0's l1: 0.465508
[300]	valid_0's l1: 0.447319
[400]	valid_0's l1: 0.438655
[500]	valid_0's l1: 0.433666
[600]	valid_0's l1: 0.430811
[700]	valid_0's l1: 0.428915
[800]	valid_0's l1: 0.427561
[900]	valid_0's l1: 0.426588
[1000]	valid_0's l1: 0.425851
[1100]	valid_0's l1: 0.425412
[1200]	valid_0's l1: 0.425138
[1300]	valid_0's l1: 0.424947
[1400]	valid_0's l1: 0.424815
[1500]	valid_0's l1: 0.424752
[1600]	valid_0's l1: 0.424752
Early stopping, best iteration is:
[1476]	valid_0's l1: 0.42475


[I 2025-12-16 20:59:58,686] Trial 1 finished with value: 1163.6001971997146 and parameters: {'num_leaves': 99, 'max_depth': 7, 'min_data_in_leaf': 328, 'learning_rate': 0.010074238942079334, 'feature_fraction': 0.7099025726528951, 'feature_fraction_bynode': 0.7159725093210578, 'bagging_fraction': 0.6873687420594126, 'bagging_freq': 7, 'lambda_l1': 1.3949386065204183, 'lambda_l2': 14.607232426760907, 'min_gain_to_split': 1.8318092164684585, 'max_bin': 255}. Best is trial 0 with value: 1162.2211620581852.


...

Training until validation scores don't improve for 200 rounds
[100]	valid_0's l1: 0.519777
[200]	valid_0's l1: 0.467155
[300]	valid_0's l1: 0.445639
[400]	valid_0's l1: 0.435673
[500]	valid_0's l1: 0.430281
[600]	valid_0's l1: 0.427286
[700]	valid_0's l1: 0.425576
[800]	valid_0's l1: 0.424407
[900]	valid_0's l1: 0.423637
[1000]	valid_0's l1: 0.423102
[1100]	valid_0's l1: 0.422716
[1200]	valid_0's l1: 0.422463
[1300]	valid_0's l1: 0.42235
[1400]	valid_0's l1: 0.422227
[1500]	valid_0's l1: 0.422157
[1600]	valid_0's l1: 0.422096
[1700]	valid_0's l1: 0.422067
[1800]	valid_0's l1: 0.422058
[1900]	valid_0's l1: 0.422058
Early stopping, best iteration is:
[1760]	valid_0's l1: 0.422053


[I 2025-12-16 21:59:13,469] Trial 18 finished with value: 1157.8246963258237 and parameters: {'num_leaves': 213, 'max_depth': 12, 'min_data_in_leaf': 262, 'learning_rate': 0.007769969817539154, 'feature_fraction': 0.7615645284400153, 'feature_fraction_bynode': 0.8943720345909655, 'bagging_fraction': 0.7721047063967424, 'bagging_freq': 8, 'lambda_l1': 0.14140474515899223, 'lambda_l2': 9.506593017393062, 'min_gain_to_split': 1.3878041045756073, 'max_bin': 255}. Best is trial 12 with value: 1152.6475239390618.


Training until validation scores don't improve for 200 rounds
[100]	valid_0's l1: 0.472496
[200]	valid_0's l1: 0.440257
[300]	valid_0's l1: 0.431065
[400]	valid_0's l1: 0.427633
[500]	valid_0's l1: 0.425942
[600]	valid_0's l1: 0.425219
[700]	valid_0's l1: 0.424954
[800]	valid_0's l1: 0.424877
[900]	valid_0's l1: 0.424852
[1000]	valid_0's l1: 0.424852
Early stopping, best iteration is:
[844]	valid_0's l1: 0.424852


[I 2025-12-16 22:01:01,543] Trial 19 finished with value: 1166.372416866214 and parameters: {'num_leaves': 251, 'max_depth': 10, 'min_data_in_leaf': 500, 'learning_rate': 0.015753242561424064, 'feature_fraction': 0.8281848300150375, 'feature_fraction_bynode': 0.685840191091291, 'bagging_fraction': 0.8303921550345197, 'bagging_freq': 5, 'lambda_l1': 3.606445241280038, 'lambda_l2': 22.71013606420069, 'min_gain_to_split': 2.2686329296463446, 'max_bin': 255}. Best is trial 12 with value: 1152.6475239390618.


Best MAE:  1152.6475239390618
Best params:  {'num_leaves': 236, 'max_depth': 9, 'min_data_in_leaf': 203, 'learning_rate': 0.015748462517233382, 'feature_fraction': 0.7604063212686378, 'feature_fraction_bynode': 0.7966713343930665, 'bagging_fraction': 0.8106707103422641, 'bagging_freq': 7, 'lambda_l1': 2.721190423876234, 'lambda_l2': 18.412028624412162, 'min_gain_to_split': 0.17783197825737135, 'max_bin': 255}


In [ ]:
'''
Run 20 trials
Best MAE:  1152.6475239390618
Best params:  
{'num_leaves': 236, 
 'max_depth': 9, 
 'min_data_in_leaf': 203, 
 'learning_rate': 0.015748462517233382, 
 'feature_fraction': 0.7604063212686378, 
 'feature_fraction_bynode': 0.7966713343930665, 
 'bagging_fraction': 0.8106707103422641, 
 'bagging_freq': 7, 
 'lambda_l1': 2.721190423876234, 
 'lambda_l2': 18.412028624412162, 
 'min_gain_to_split': 0.17783197825737135, 
 'max_bin': 255}
'''


'''
# Run 40 trials
Early stopping, best iteration is:
[1400]	valid_0's l1: 0.416135
[I 2025-12-13 01:03:05,485] Trial 36 finished with value: 1140.1649585398006 and parameters: {'num_leaves': 404, 'max_depth': 8, 'min_data_in_leaf': 202, 'learning_rate': 0.02361306858113298, 'feature_fraction': 0.7211579800007945, 'feature_fraction_bynode': 0.9081132434036039, 'bagging_fraction': 0.8727967232517579, 'bagging_freq': 7, 'lambda_l1': 5.763583420124947, 'lambda_l2': 14.460054311753241, 'min_gain_to_split': 0.032912218537174215, 'max_bin': 127}. Best is trial 36 with value: 1140.1649585398006.

# Best params 
# MAE: 1140.1649585398006 
# trial 36 (40 trials)
{'num_leaves': 404, 
 'max_depth': 8, 
 'min_data_in_leaf': 202, 
 'learning_rate': 0.02361306858113298, 
 'feature_fraction': 0.7211579800007945, 
 'feature_fraction_bynode': 0.9081132434036039, 
 'bagging_fraction': 0.8727967232517579, 
 'bagging_freq': 7, 
 'lambda_l1': 5.763583420124947, 
 'lambda_l2': 14.460054311753241, 
 'min_gain_to_split': 0.032912218537174215, 
 'max_bin': 127}
'''
